<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/KoBERTopic_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0_newDiscriminataion_Okt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install bertopic[visualization]

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install bertopic

In [ ]:
%pip install bertopic[visualization]

# 필요한 것들을 임포트

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from bertopic import BERTopic

# 데이터 불러오기

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
documents = [line.strip() for line in open("newDiscrimination_text.txt", encoding="utf-8").readlines()]
documents[:5]

['못사는 놈들은 정말 불쌍해.',
 '쟤처럼 느글거리는 인상이 꼭 사고치더라',
 '어린 애들 따 먹어봤다는 거잖아 좀 부럽네',
 '너 계속 이러면 나중에 청소차 몰고 다닌다고 엄마가 말했어 안했어',
 '와 미국인 정서 걍 이해불가던데 어떻게 사귀는거지?']

# Mecab과 SBERT를 이용한 Bertopic (형태소단위)

In [ ]:
# 불용어 사전 만들기
korean_stopwords_path = "./korean_stopwords.txt"

with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []  # 불용어 제거
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

## 토픽개수 8개로 지정

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(documents)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.04808647064320864), ('ㅋㅋ', 0.047055562424622784), ('ㅋㅋㅋ', 0.045818898090979096), ('남자', 0.03130009873860803), ('니까', 0.02624033241659972), ('하는', 0.025083155904808845), ('ooo', 0.02458491959595833), ('사람', 0.024582928009412294), ('맞는', 0.02254479804041785), ('하고', 0.0211408191346334)]
1 번째 토픽 : [('전라도', 0.07176562821895463), ('하는게', 0.04807671984457218), ('다운증후군', 0.044151733191726904), ('이면', 0.04145451441534167), ('한테', 0.03995313651050213), ('처럼', 0.03886313091382898), ('이나', 0.036192944237399445), ('나와서', 0.034834814132551566), ('환자', 0.0342984022599936), ('여자', 0.03389188757751242)]
2 번째 토픽 : [('한국', 0.2268036977954156), ('조선족', 0.14485592992714372), ('서울', 0.11855521070999382), ('한국인', 0.09253092506757939), ('북한', 0.07326760387272128), ('사람', 0.053495530227918725), ('인천', 0.047274378681190936), ('대한민국', 0.03676302545860291), ('이라고', 0.03360327584491158), ('동네', 0.032376573703202155)]
3 번째 토픽 : [('그건', 0.07486381804990522), ('...', 0.07409450803150462), ('수준', 

In [ ]:
model.visualize_barchart()

## 토픽개수 지정 안함

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=10,
                 calculate_probabilities=True)

topics, probs = model.fit_transform(documents)

for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.08935008538486194), ('남자', 0.062222155276737356), ('하는', 0.04114553495971082), ('사람', 0.039973513080359714), ('ㅋㅋ', 0.03204016101635464), ('하고', 0.02612137028761119), ('한테', 0.02514492020310094), ('이야', 0.023980543170870068), ('ㅋㅋㅋ', 0.023749961024996314), ('하면', 0.023425220814554178)]
1 번째 토픽 : [('한국', 0.1885052653974335), ('조선족', 0.16623441576182327), ('중국', 0.15471498647731882), ('중국인', 0.11868327089395125), ('서울', 0.08996452164638155), ('사람', 0.06606587347415406), ('한국인', 0.06067813110788686), ('일본', 0.051388132559554774), ('북한', 0.050389758504938195), ('여자', 0.04366717146691712)]
2 번째 토픽 : False


In [ ]:
model.visualize_barchart(top_n_topics = 20)

## nr_topics="auto" 자동 토픽 축소 사용 (모델이 자동으로 토픽의 수를 줄이도록 설정)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('남자', 0.021227113027142), ('ㅋㅋ', 0.02056184926386971), ('전라도', 0.020426083283783654), ('ㅋㅋㅋ', 0.016438971509081118), ('하는', 0.016153750914832992), ('짱깨', 0.015145556618968025), ('여자', 0.014469736564291933), ('사람', 0.014032710884238879), ('이나', 0.013683251663688459), ('맞아', 0.01367469580239628)]
1 번째 토픽 : [('ㅋㅋ', 0.029118700978235233), ('여자', 0.018286225652113675), ('남자', 0.01636405942698146), ('이나', 0.015973395889505793), ('하고', 0.01585129123858157), ('아이돌', 0.01543271405673601), ('이라고', 0.014350897066877397), ('하는데', 0.014198198934661828), ('사람', 0.014132923845352328), ('회사', 0.014058880830149821)]
2 번째 토픽 : [('한국', 0.13807414211438465), ('조선족', 0.10501289949537422), ('서울', 0.09079404360486723), ('한국인', 0.056336240653623984), ('북한', 0.044850409537862866), ('인천', 0.03743430400837374), ('한국어', 0.03134432605474738), ('사람', 0.026952665205504156), ('대한민국', 0.023508244541060538), ('이제', 0.021676236957308043)]
3 번째 토픽 : [('여자', 0.04716417817420624), ('않는', 0.03606876732941503), (

In [ ]:
model.visualize_barchart(top_n_topics = 20)

- 명사가 아닌 단어들은 의미가 없어보인다.

# 명사단위 분석

## 토픽개수 8개로 설정

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.nouns(sent)

        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
           if w not in stopwords: 
             result.append(w)
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Okt())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('남자', 0.0591774640011451), ('짱깨', 0.05415663763346857), ('사람', 0.04378471062420949), ('게이', 0.038229048613390146), ('전라도', 0.03744561972064772), ('여자', 0.03490375575739441), ('장애인', 0.028819457862667756), ('그게', 0.025134417870400768), ('수준', 0.021130735119619354), ('홍어', 0.02079635856465576)]
1 번째 토픽 : [('한국', 0.18980519791794082), ('조선족', 0.166785740865046), ('중국', 0.15947052069930928), ('중국인', 0.12398090207872041), ('서울', 0.09196358772530834), ('한국인', 0.06799862862366643), ('북한', 0.06305231608241935), ('일본', 0.05618934858053197), ('사람', 0.052299402640110626), ('일본인', 0.037090161067454414)]
2 번째 토픽 : [('여자', 0.059533422118459016), ('남자', 0.046355680134230325), ('큰일', 0.03111181273750288), ('차이', 0.030544651641838148), ('사람', 0.02991106685084078), ('방송인', 0.02720043734800185), ('길거리', 0.025548170902052435), ('수준', 0.024828613765552742), ('가지', 0.024316078194939927), ('숏컷', 0.024316078194939927)]
3 번째 토픽 : [('여자', 0.28652815711589946), ('백인', 0.2305238022298866), ('피부', 0.07

In [ ]:
model.visualize_barchart()

## 토픽 개수 미설정

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('회사', 0.018560045406400195), ('출신', 0.015985622640161987), ('취직', 0.015775058053552466), ('아이돌', 0.015637610617995346), ('여자친구', 0.01525565222546627), ('여친', 0.015091848747234876), ('남자', 0.014899473172263603), ('여자', 0.014892648413082719), ('사람', 0.014851060946293213), ('연예인', 0.01399659185736139)]
1 번째 토픽 : [('여자', 0.021493082257746526), ('길거리', 0.019985957324079148), ('큰일', 0.017555324858478268), ('남자', 0.01739207715536718), ('방송인', 0.01636217977728837), ('주제', 0.015757865925311355), ('애초', 0.014989467993059362), ('가지', 0.013972307664995293), ('숏컷', 0.013972307664995293), ('차선', 0.013426968149317651)]
2 번째 토픽 : [('전라도', 0.05035493396633978), ('다운증후군', 0.034702936710444485), ('한남', 0.030897052723129575), ('알바', 0.02927492964943786), ('이면', 0.02662540545211337), ('거기', 0.025750965064635492), ('환가', 0.02523707625971224), ('구석', 0.02409465655864097), ('환자', 0.02349642961443456), ('육아휴직', 0.02313529114029633)]
3 번째 토픽 : [('한국', 0.15174314406416617), ('조선족', 0.1076170888894801

In [ ]:
model.visualize_barchart(top_n_topics=10)

## nr_topics="auto" 자동 토픽 축소 사용

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.10485351043672617), ('남자', 0.0784982034368717), ('사람', 0.053436776625590904), ('장애인', 0.026752159344369753), ('흑인', 0.026490310748537207), ('얼굴', 0.025149374696976784), ('생각', 0.02415985020938909), ('결혼', 0.023338519255382197), ('전라도', 0.0224946787293356), ('게이', 0.022116097829477786)]
1 번째 토픽 : [('한국', 0.32776699795667463), ('조선족', 0.2615515035708252), ('서울', 0.19043904332561243), ('한국인', 0.1129556276466036), ('북한', 0.10212747503574976), ('사람', 0.09196885468407348), ('인천', 0.07166811725396671), ('한국어', 0.05935520948358088), ('나라', 0.04530704431332241), ('대한민국', 0.04451640711268566)]
2 번째 토픽 : [('중국', 0.6084107906792623), ('중국인', 0.4844253749273391), ('일본', 0.12089237492627869), ('조선족', 0.09425086866704847), ('베트남', 0.08987385699241512), ('살수', 0.06859089424411702), ('이중국적자', 0.06859089424411702), ('중국어', 0.06859089424411702), ('개국', 0.0647510345125535), ('짱꼴라', 0.0647510345125535)]
3 번째 토픽 : False


In [ ]:
model.visualize_barchart()

- BERTopic을 적용하니 토픽의 개수가 2-4개로 나오는 경우도 꽤 있다.